# Introduction to SciPy
Tutorial at EuroSciPy 2019, Bilbao
## 2. Signal analysis – Rocking motion of a TGV

In [ ]:
import numpy as np
from scipy import fftpack, signal
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

### Importing the data

In [ ]:
data = np.genfromtxt('data/TGV_data.csv.bz2', delimiter=',', names=True)
time = data['Time_s']
omega_y = data['Gyroscope_y_rads']
_ = plt.plot(time, omega_y)

### Time spacing

Before analyzing the data any further, let us take a look at the time elapsed between subsequent measurements.

In [ ]:
time_intervals = time[1:]-time[:-1]

In [ ]:
_ = plt.hist(time_intervals, bins=100)

We will ignore these small differences and assume the data to be equally spaced in time with the mean time difference between subsequent data points.

In [ ]:
delta_t = np.mean(time_intervals)
delta_t

### Spectral analysis

Fourier transformation of a discrete real signal with `rfft`:
$$y_j = \sum_{k=0}^{n-1} x_k \exp\left(-\text{i}\frac{2\pi jk}{n}\right)$$
With exception of $y_0$ and, for even $n$, $y_{n/2}$ all $y_j$ are complex.
`rfft` returns an array $y_0, \text{Re}(y_1), \text{Im}(y_1),\ldots$

In [ ]:
fft_y = fftpack.rfft(omega_y)

In [ ]:
_ = plt.plot(fft_y[::2])

In [ ]:
_ = plt.plot(fft_y[1::2])

In [ ]:
freqs = fftpack.rfftfreq(omega_y.shape[0], delta_t)

In [ ]:
_ = plt.plot(freqs[::2], fft_y[::2])

In [ ]:
_ = plt.plot(freqs[1::2], fft_y[1::2])

<div style="color: DarkBlue;background-color: LightYellow">
<i>Exercise:</i> Given the Fourier transform, how does one get back to the original data?
</div>

### Power spectral density

In [ ]:
fft_y = np.insert(fft_y, 0, 0)
fft_square = fft_y*fft_y
power = fft_square[::2]+fft_square[1::2]
power = 2*delta_t/fft_square.shape[0]*power
_ = plt.plot(freqs[::2], power)

In [ ]:
_ = plt.plot(*signal.periodogram(omega_y, 1/delta_t))

### Filter out the rocking signal

We are interested in the signal around 1.4 Hz. Filter out frequencies beyond 3 Hz.

In [ ]:
filter_coeffs = signal.firwin(301, 3, pass_zero=True, fs=1/delta_t)

In [ ]:
_ = plt.plot(filter_coeffs, '.')

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
_ = plt.plot(freqs, 20*np.log10(np.abs(response)))

In [ ]:
_ = plt.polar(np.angle(response), np.abs(response))

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
omega_y.shape, omega_y_filtered.shape

In [ ]:
_ = plt.plot(time[150:-150], omega_y[150:-150])

In [ ]:
_ = plt.plot(time[150:-150], omega_y_filtered)

Filter out anything but the range from 53-70 Hz

In [ ]:
filter_coeffs = signal.firwin(301, [53, 70], pass_zero='bandpass', fs=1/delta_t)

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
_ = plt.plot(freqs, 20*np.log10(abs(response)))

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
_ = plt.plot(time[150:-150], omega_y[150:-150])
_ = plt.plot(time[150:-150], omega_y_filtered)

In [ ]:
_ = plt.plot(fftpack.rfftfreq(omega_y_filtered.shape[0], delta_t)[::2],
             fftpack.rfft(omega_y_filtered)[::2])

### Spectrogram

In [ ]:
freq, sp_time, Sxx = signal.spectrogram(omega_y, fs=1/delta_t)
_ = plt.pcolormesh(sp_time, freq, Sxx)

<div style="color: DarkBlue;background-color: LightYellow">
<i>Exercise:</i> Try to obtain more structure by plotting the logarithm of the spectrogram.
</div>

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
_ = ax.plot_surface(sp_time, freq[:, np.newaxis], Sxx, cmap=cm.viridis)